In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
# unmount google drive directory code 
# drive.flush_and_unmount()

In [0]:
import os
import json
import gzip
import pandas as pd
from urllib.request import urlopen

In [0]:
# you can either download the file from here or directly open it after mounting google drive

#!wget http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/Electronics_5.json.gz
#!wget https://drive.google.com/file/d/18wfGKYb0dgc6sUQn9Szylc6Qn5gPHdkN

In [3]:
# set working directory -> you must set the path into which you have uploaded the zipped file
%cd /content/drive/My\ Drive/SNLP\ Project

/content/drive/My Drive/SNLP Project


In [4]:
# list content of drive
%ls

 Electronics_5.json.gz  'Project Ideas.gdoc'  'Project Proposal.gdoc'


In [5]:
data = []
with gzip.open('Electronics_5.json.gz') as f:
    for l in f:
        data.append(json.loads(l.strip()))
    
# total length of list, this number equals total number of products
print(len(data))

# first row of the list
print(data[0])

6739590
{'overall': 5.0, 'vote': '67', 'verified': True, 'reviewTime': '09 18, 1999', 'reviewerID': 'AAP7PPBU72QFM', 'asin': '0151004714', 'style': {'Format:': ' Hardcover'}, 'reviewerName': 'D. C. Carrad', 'reviewText': 'This is the best novel I have read in 2 or 3 years.  It is everything that fiction should be -- beautifully written, engaging, well-plotted and structured.  It has several layers of meanings -- historical, family,  philosophical and more -- and blends them all skillfully and interestingly.  It makes the American grad student/writers\' workshop "my parents were  mean to me and then my professors were mean to me" trivia look  childish and silly by comparison, as they are.\nAnyone who says this is an  adolescent girl\'s coming of age story is trivializing it.  Ignore them.  Read this book if you love literature.\nI was particularly impressed with  this young author\'s grasp of the meaning and texture of the lost world of  French Algeria in the 1950\'s and \'60\'s...parti